In [1]:
import geojson
from datetime import datetime
import pandas as pd
import requests

In [ ]:
pd.read_csv("../data/54152.csv", encoding="latin-1", sep=";")

In [ ]:
!ls

In [ ]:

with open("../data/parcours_navir/CAMPAGNE_LIGNE.geojson") as f:
    gj = geojson.load(f)
features = gj['features'][0]

In [ ]:
len(gj['features'])

In [ ]:
# format
format = '%Y-%m-%d'

features = [x for x in gj["features"] if datetime.strptime(x["properties"]["START_DATE"], format).year == 2021]

In [ ]:
gj["features"][0]["properties"]["START_DATE"]

In [ ]:

  
# convert from string format to datetime format
date = datetime.strptime(input, format)

In [ ]:
date.year == 2022

In [ ]:
gj.keys()

In [ ]:
with open("../data/emission_impulsive_forte_2016/DCSMM2018_D11C1.3.geojson") as f:
    gj = geojson.load(f)
features = gj['features'][0]

In [ ]:
gj['features'][500]

In [ ]:
gj.keys()

In [ ]:
min_longitude = -2.250511
max_longitude = -0.802813
min_latitude = 45.670575
max_latitude = 46.378693
top_left = (max_latitude, min_longitude)
top_right = (max_latitude, max_longitude)
bottom_left = (min_latitude, min_longitude)
bottom_right = (min_latitude, max_longitude)

# Position des bateaux

In [ ]:
api_key = "af3f41ea705f7881e533cdac088cfe72c154be34"

In [ ]:
url = "https://services.marinetraffic.com/api/exportvesseltrack"
url

In [ ]:
headers = {'Content-Type': 'application/json'}
res = requests.get(url, headers=headers, params={"api_key": api_key})
res

In [ ]:
res.text

In [ ]:
from marinetrafficapi import MarineTrafficApi
from tqdm.notebook import tqdm

api = MarineTrafficApi(api_key="af3f41ea705f7881e533cdac088cfe72c154be34")

In [ ]:
int(max_latitude - min_latitude)

In [ ]:
la_rochelle_vessel_position = []
current_lat, current_long = min_latitude, min_longitude
for lat_delta in tqdm(range(int(max_latitude - min_latitude) + 1)):
    for long_delta in range(int(max_longitude - min_longitude) + 1):
        vessel_positions = api.vessel_historical_track(period='daily', 
                                                       days=3, 
                                                       mmsi=241486000,
                                                       min_longitude=current_long,
                                                       max_longitude=current_long + long_delta,
                                                       min_latitude=current_lat,
                                                       max_latitude=current_lat + lat_delta)
        current_lat, current_long = current_lat + lat_delta, current_long + long_delta
        la_rochelle_vessel_position += vessel_positions.to_list

In [ ]:
vessel_positions = api.vessel_historical_track(period='daily', 
                                               days=3, 
                                               min_longitude=min_longitude,
                                               max_longitude=min_longitude+1,
                                               min_latitude=min_latitude,
                                               max_latitude=min_latitude+1)

In [ ]:
vessel_positions.to_list

In [ ]:
MarineTrafficApi.print_params_for('vessel_historical_track')

In [ ]:
response = api.__api_call_method__(protocol='json'|'jsono'|'csv'|'xml', # default is jsono
                                   msg_type='simple'|'extended',  # default is simple
                                   timeout=10) # default is 5 (5 seconds)

In [ ]:
import sys
import importlib
import numpy as np
import math
from math import pi, sin, acos, cos
sys.path.insert(0, "/home/mqueyrel/Documents/deseabel/deseabel/ocean_ecosystem/")
import noise_impactor
importlib.reload(noise_impactor)

In [ ]:
from noise_impactor import NoiseImpactor, Boat, FishingBoat, PileDriving

In [ ]:
maille_TL = compute_decibels_matrix(45.8, -1.8)

In [ ]:
SLi = compute_SLi(10, 30)

In [ ]:
min_longitude = -2.250511
max_longitude = -0.802813
min_latitude = 45.670575
max_latitude = 46.378693

range_lat = np.arange(min_latitude, max_latitude, 0.01)
range_long = np.arange(min_longitude, max_longitude, 0.0015)

In [ ]:
df_res = pd.DataFrame(RL)
df_res.columns = range_long
df_res["lat"] = range_lat

In [ ]:
df_res = df_res.melt(id_vars=["lat"], 
        var_name="lon", 
        value_name="value")

In [ ]:
RL

In [ ]:
df_res

In [ ]:
import geopandas
gdf = geopandas.GeoDataFrame(
    df_res, geometry=geopandas.points_from_xy(df_res.lon, df_res.lat))

In [ ]:
gdf.to_file("../data/heatmap.geojson", driver="GeoJSON")

In [ ]:
gdf.plot()

In [ ]:
RL = update_decibels_matrix_with_speed(maille_TL, SLi, 1)

In [ ]:
RL

In [ ]:
RL2 = update_decibels_matrix_with_speed(maille_TL, SLi, 2)
RL3 = update_decibels_matrix_with_speed(maille_TL, SLi, 3)

In [ ]:
import seaborn

In [ ]:
seaborn.heatmap(RL)

In [ ]:
RL / RL2

In [ ]:
seaborn.heatmap(RL2)

In [ ]:
seaborn.heatmap(RL3)

In [ ]:
np.arccos

In [ ]:
fish = geopandas.read_file("../data/Fish.geojson")

In [ ]:
fish.plot()

In [ ]:
fish

In [1]:
import sys
import importlib
import numpy as np
import math
from math import pi, sin, acos, cos
sys.path.insert(0, "/home/mqueyrel/Documents/deseabel/deseabel/ocean_ecosystem")
import noise_impactor
importlib.reload(noise_impactor)
from noise_impactor import FishingBoat, Cargo, PileDriving
from simulator import Simulator
import pandas as pd
import geopandas
import seaborn
from utils import load_config

In [2]:
config = load_config()
boat1 = FishingBoat(lat=45.70, long=-2.1, speed=5)
boat2 = FishingBoat(lat=45.85, long=-1.75, speed=10)
boat3 = FishingBoat(lat=46, long=-1.9, speed=7)

In [ ]:
simu = Simulator(noise_impactors=[boat1, boat2, boat3], marine_fauna_files=None)

In [ ]:
hm_total = simu.compute_and_add_heatmaps()

In [ ]:
hm_total_quantified = hm_total.copy()

val_dead = 62
val_mean = 55
val_zen = 51

hm_total_quantified[hm_total_quantified < val_zen] = 0
hm_total_quantified[(hm_total_quantified <= val_mean) & (hm_total_quantified >= val_zen)] = 1
hm_total_quantified[(hm_total_quantified <= val_dead) & (hm_total_quantified >= val_mean)] = 2
hm_total_quantified[hm_total_quantified > val_dead] = 3

In [ ]:
df_res = pd.DataFrame(hm_total_quantified[::-1])
df_res.columns = boat1.range_long
df_res["lat"] = boat1.range_lat
df_res = df_res.melt(id_vars=["lat"], 
        var_name="lon", 
        value_name="value")

In [ ]:
gdf = geopandas.GeoDataFrame(
        df_res, geometry=geopandas.points_from_xy(df_res.lon, df_res.lat))

In [ ]:
gdf[gdf["value"].isin([1,2,3])].to_file("../data/safe_zone_noeuds.geojson", driver="GeoJSON")
gdf[gdf["value"].isin([2,3])].to_file("../data/hurt_zone_noeuds.geojson", driver="GeoJSON")
gdf[gdf["value"] == 3].to_file("../data/dead_zone_noeuds.geojson", driver="GeoJSON")

In [ ]:
seaborn.heatmap(hm_total)

In [ ]:
# min_longitude = -2.250511
# max_longitude = -0.802813
# min_latitude = 45.670575
# max_latitude = 46.378693

speed = 10
lat = 45.9
long=-1.9
boat1 = FishingBoat(lat=45.70, long=-2.1, speed=10)
boat2 = FishingBoat(lat=lat, long=long, speed=speed)
boat3 = FishingBoat(lat=46, long=-1.9, speed=10)

simu = Simulator(noise_impactors=[boat1, boat2, boat3], marine_fauna_files=None)
simu = Simulator(noise_impactors=[boat2], marine_fauna_files=None)

hm_total = simu.compute_and_add_heatmaps()

hm_total_quantified = hm_total.copy()

val_dead = 62
val_mean = 55
val_zen = 51

hm_total_quantified[hm_total_quantified < val_zen] = 0
hm_total_quantified[(hm_total_quantified <= val_mean) & (hm_total_quantified >= val_zen)] = 1
hm_total_quantified[(hm_total_quantified <= val_dead) & (hm_total_quantified >= val_mean)] = 2
hm_total_quantified[hm_total_quantified > val_dead] = 3

print("lat", lat, "long", long)

In [ ]:
seaborn.heatmap(hm_total)

In [ ]:
seaborn.heatmap(hm_total_quantified[::-1])

In [ ]:
# df_res = pd.DataFrame(hm_total)
# df_res.columns = boat1.range_long
# df_res["lat"] = boat1.range_lat
# df_res = df_res.melt(id_vars=["lat"], 
#         var_name="lon", 
#         value_name="value")

# gdf = geopandas.GeoDataFrame(
#     df_res, geometry=geopandas.points_from_xy(df_res.lon, df_res.lat))

# gdf.to_file("../data/decibels.geojson", driver="GeoJSON")

In [ ]:
df_res = pd.DataFrame(hm_total_quantified[::-1])
df_res.columns = boat1.range_long
df_res["lat"] = boat1.range_lat
df_res = df_res.melt(id_vars=["lat"], 
        var_name="lon", 
        value_name="value")

gdf = geopandas.GeoDataFrame(
    df_res, geometry=geopandas.points_from_xy(df_res.lon, df_res.lat))

In [ ]:
gdf.to_file("../data/decibels_quantified.geojson", driver="GeoJSON")

In [ ]:
gdf[gdf["value"].isin([1,2,3])].to_file(f"../data/safe_zone_{speed}_noeuds.geojson", driver="GeoJSON")
gdf[gdf["value"].isin([2,3])].to_file(f"../data/hurt_zone_{speed}_noeuds.geojson", driver="GeoJSON")
gdf[gdf["value"] == 3].to_file(f"../data/dead_zone_{speed}_noeuds.geojson", driver="GeoJSON")

In [ ]:
gdf[gdf["value"] == 1].to_file("../data/safe_zone_hole.geojson", driver="GeoJSON")
gdf[gdf["value"] == 2].to_file("../data/hurt_zone_hole.geojson", driver="GeoJSON")